Comando abaixo apenas para que seja possível importar os modulos corretamente

In [1]:
import sys
sys.path.append("../") 

# Extração Simples de um único texto

Vamos supor que desejamos extrair as features do texto abaixo:

In [2]:
texto = "<h1> oioi</h1>meu nome é Daniel. Tudo bem? :)"

Caso desejamos extrair o número de palavras e caracteres no texto, fazemos:
1. Imports necessários
    - `WordCountFeature` e `CharacterCountFeature`: Features que calculam a quantidade de palavras e caracteres no texto
    - `FormatEnum`: Formato do texto a ser processado (HTML ou text_plain)
    - `FeatureTimePerDocumentEnum`: Apenas especifica o tempo medio para extrair uma determinada feature
    - `FeatureVisibilityEnum`: Verifica se a feature é publica ou não
    - `FeatureCalculator`: é a superclasse de todas as features. Dentro dela, há um objeto featManager que é responsável por preprocessar as features
    - `Document`: Representa o documento a ser processado

In [3]:
from feature.featureImpl.style_features import WordCountFeature,CharacterCountFeature
from utils.basic_entities import FormatEnum, FeatureTimePerDocumentEnum
from feature.features import  FeatureVisibilityEnum,FeatureCalculator,Document

2. Criamos as features:

In [4]:


#criamos a feature
word_count = WordCountFeature("Word Count","Count the number of words in the text.","",
                    FeatureVisibilityEnum.public, FormatEnum.text_plain, 
                              FeatureTimePerDocumentEnum.MILLISECONDS,ignore_punctuation=True)

charCountFeat = CharacterCountFeature("Char Count","Count the number of characters in the text.","",
                    FeatureVisibilityEnum.public, FormatEnum.text_plain, 
                    FeatureTimePerDocumentEnum.MILLISECONDS, ignore_punctuation=False)
    
arr_features = [word_count,charCountFeat]



2. Instanciamos o documento:

In [5]:
docText = Document(1,"Documento bonitinho",texto)



3. executamos o o metodo para calcular as features passando o quais features temos que calcular e o formato do texto (HTML ou text_plain)


In [6]:
arr_resultado = FeatureCalculator.featureManager.computeFeatureSet(docText,arr_features,FormatEnum.HTML)

O resultado, para cada posição `i` do vetor de features `arr_features`, temos o resultado na posição `i` do vetor `arr_resultados`:

In [7]:
for i,feature in enumerate(arr_features):
    print("{nome_feature}: {resultado}".format(nome_feature=feature.name,resultado=arr_resultado[i]))

Word Count: 8
Char Count: 36


# Uso do FeatureFactory para extração de diversas features

Como foi possível observar, cada feature deve ser instanciada e configurada para ser usada (por exemplo, o número de caracteres você configura se pode ignorar ou não pontuação). Por causa disso, caso desejamos extrair todas sas features textuais, isso pode ser um processo muito trabalhoso.

Pensando nisso, criamos o FeatureFactory, que possui todas as features desenvolvidas com parametros padrão. Você pode instanciar todas as features textuais da seguinte forma:

Import necessario: 
    - `FeatureFactory`: A fabrica da features préconfiguradas
    - `LanguageEnum`: Idioma - é importante especificar para algumas features

In [8]:
from feature.feature_factory.feature_factory import FeatureFactory 
from utils.basic_entities import LanguageEnum

Existem várias factories de features todas subclasses de `FeatureFactory`. Tais classes implementam o método `createFeatures` que retornam as features instanciadas. 

In [16]:
#algumas features são dependentes de idioma, assim, você deve especificar o idiom
objEnglish = LanguageEnum.en
arr_features = []
for SubClass in FeatureFactory.__subclasses__():
    objFeatFact = None
    
    if(SubClass.IS_LANGUAGE_DEPENDENT):
        objFeatFact = SubClass(objEnglish)
    else:
        objFeatFact = SubClass()
        
    #caso a feature não esteja em desenvolvimento, anexar (podem ter features que ainda não fizemos os testes)
    if(not SubClass.DEVELOPMENT):
        arr_features.extend(objFeatFact.createFeatures())

In [17]:
for i,feature in enumerate(arr_features):
    print("{i}: {name}: {description}".format(i=i,name=feature.name,description=feature.description))

0: Section count: Count the number of sections (i.e. HTML h1 tags) in the text
1: Subsection count: Count the number of subsections (i.e. HTML h1 tags) in the text
2: Complete URL link count: Count the number of  HTML 'a' tag in which the 'href' attribute refers to a complete URL.
3: Complete URL link count per section: Ration between number of  HTML 'a' tag in which the 'href' attribute refers to a complete URL and the number of sections.
4: Complete URL link count per length: Ration between number of  HTML 'a' tag in which the 'href' attribute refers to a complete URL and the number of characters in text.
5: Relative URL link count: Count the number of  HTML 'a' tag in which the 'href' attribute refers to a relative URL (e.g. /images/cow.gif).
6: Relative URL link count per section: Ratio between the number of  HTML 'a' tag in which the 'href' attribute refers to a relative URL (e.g. /images/cow.gif) and the number of sections.
7: Relative URL link count per length: Ratio between the

Podemos, assim, calcular o resultado para todas as features dado um texto:

In [18]:
arr_resultado = FeatureCalculator.featureManager.computeFeatureSet(docText,arr_features,FormatEnum.HTML)
for i,feature in enumerate(arr_features):
    print("{nome_feature}: {resultado}".format(nome_feature=feature.name,resultado=arr_resultado[i]))

Section count: 1
Subsection count: 0
Complete URL link count: 0
Complete URL link count per section: 0.0
Complete URL link count per length: 0.0
Relative URL link count: 0
Relative URL link count per section: 0.0
Relative URL link count per length: 0.0
Same page link count: 0
Same page link count per section: 0.0
Same page link count per length: 0.0
Mean section size: 31.0
Mean subsection size: 0
Largest section size: 31
Shortest section size: 31
Standard deviation of the section size: 0
Images count: 0
Images per length: 0.0
Images per section: 0.0
Images per subsection: 0.0
Phrase count: 3
Large phrase count: 0
Paragraph count: 1
Large paragraph count: 0
Largest phrase size: 5
Large phrase rate: 0.3333333333333333
Short phrase rate: 0.0
Char Count: 36
Word Count: 8
Articles count: 0
Auxiliary verbs count: 0
Coordination conjunctions count: 0
Correlative conjunctions count: 0
Indefinite pronouns count: 0
Interrogative pronouns count: 0
Prepositions count: 0
Pronouns count: 0
Relative 